In [1]:
import json
import pandas as pd
import shapely
import geopandas as gpd
from shapely.geometry import LineString, Point
import pandana
import osmnet

In [2]:
lat_min = 42.904155
long_min = -2.621987
lat_max = 43.403070
long_max = -1.740334

## Drive

In [3]:
drive_nodes, drive_edges = osmnet.load.network_from_bbox(
    lat_min=lat_min, lng_min=long_min, lat_max=lat_max, 
    lng_max=long_max, network_type='drive', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](42.90415500,-2.62199057,43.15677470,-2.17936200);>;);out;'}"
Downloaded 12,947.8KB from www.overpass-api.de in 1.73 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](43.15445613,-2.62199057,43.40391960,-2.17600228);>;);out;'}"
Downloaded 9,802.2KB from www.ove

In [4]:
drive_nodes=drive_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [5]:
# Add lat and long to each edge

drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
drive_edges=drive_edges.merge(drive_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [6]:

drive_edges['geometry']=drive_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [7]:
# Create geodataframe

drive_edges_gdf=gpd.GeoDataFrame(data=drive_edges, geometry='geometry', crs='epsg:4326')
drive_edges_gdf

from          to    distance  \
833859     301662479       833859   301662479   20.132225   
301662479  833859       301662479      833859   20.132225   
5462312    299850007      5462312   299850007   56.146191   
299850007  5462312      299850007     5462312   56.146191   
           299850058    299850007   299850058  368.407367   
...                           ...         ...         ...   
2496854513 303295475   2496854513   303295475  307.703954   
           2345232825  2496854513  2345232825   65.685541   
2345232825 2496854513  2345232825  2496854513   65.685541   
5171478092 5171478082  5171478092  5171478082  287.338338   
5171478082 5171478092  5171478082  5171478092  287.338338   

                                                           name      ref  \
833859     301662479                                        NaN    A-636   
301662479  833859                                           NaN    A-636   
5462312    299850007                                        NaN      NaN   
299850007  5462312                                          NaN      NaN   
           299850058                                        NaN      NaN   
...                                                         ...      ...   
2496854513 303295475                                        NaN  GI-2131   
           2345232825                              Erbeta kalea  GI-2131   
2345232825 2496854513                              Erbeta kalea  GI-2131   
5171478092 5171478082  Astrain - Irurtzun / Asterain - Irurtzun  NA-7010   
5171478082 5171478092  Astrain - Irurtzun / Asterain - Irurtzun  NA-7010   

                          highway service bridge tunnel access  ... area  \
833859     301662479     motorway     NaN    yes    NaN    NaN  ...  NaN   
301662479  833859        motorway     NaN    yes    NaN    NaN  ...  NaN   
5462312    299850007   trunk_link     NaN    NaN    NaN    NaN  ...  NaN   
299850007  5462312     trunk_link     NaN    NaN    NaN    NaN  ...  NaN   
           299850058   trunk_link     NaN    NaN    NaN    NaN  ...  NaN   
...                           ...     ...    ...    ...    ...  ...  ...   
2496854513 303295475     tertiary     NaN    NaN    NaN    NaN  ...  NaN   
           2345232825    tertiary     NaN    NaN    NaN    NaN  ...  NaN   
2345232825 2496854513    tertiary     NaN    NaN    NaN    NaN  ...  NaN   
5171478092 5171478082    tertiary     NaN    NaN    NaN    NaN  ...  NaN   
5171478082 5171478092    tertiary     NaN    NaN    NaN    NaN  ...  NaN   

                      width junction  from_lon   from_lat        id_x  \
833859     301662479    NaN      NaN -2.216077  43.044569      833859   
301662479  833859       NaN      NaN -2.216319  43.044609   301662479   
5462312    299850007    NaN      NaN -2.462793  43.076158     5462312   
299850007  5462312      NaN      NaN -2.463082  43.076616   299850007   
           299850058    NaN      NaN -2.463082  43.076616   299850007   
...                     ...      ...       ...        ...         ...   
2496854513 303295475    NaN      NaN -2.095900  43.103059  2496854513   
           2345232825   NaN      NaN -2.095900  43.103059  2496854513   
2345232825 2496854513   NaN      NaN -2.096250  43.102527  2345232825   
5171478092 5171478082   NaN      NaN -1.829623  42.904331  5171478092   
5171478082 5171478092   NaN      NaN -1.828344  42.906738  5171478082   

                         to_lon     to_lat        id_y  \
833859     301662479  -2.216319  43.044609   301662479   
301662479  833859     -2.216077  43.044569      833859   
5462312    299850007  -2.463082  43.076616   299850007   
299850007  5462312    -2.462793  43.076158     5462312   
           299850058  -2.464266  43.079813   299850058   
...                         ...        ...         ...   
2496854513 303295475  -2.094720  43.105688   303295475   
           2345232825 -2.096250  43.102527  2345232825   
2345232825 2496854513 -2.095900  43.103059  2496854513   
5171478092 51

### Ensure all roads have valid speed

In [8]:
# Delete "_link"

drive_edges_gdf['highway']=drive_edges_gdf['highway'].apply(lambda hw: hw.replace('_link', ''))

# Fill null values
# Then, change from "60 km/h" to "60"

drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].fillna(0)
drive_edges_gdf['maxspeed']=drive_edges_gdf['maxspeed'].apply(lambda s: int(str(s).split(' ')[0]))

# Group highways with type and save mode for each type

grouped = drive_edges_gdf.loc[drive_edges_gdf['maxspeed']>0].groupby('highway')['maxspeed'].agg(pd.Series.mode)
for t in drive_edges_gdf.highway.unique():
    if t not in grouped.keys():
        grouped[t] = grouped["unclassified"]
        
# Replace zero values with group mode
drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'maxspeed'] = drive_edges_gdf.loc[drive_edges_gdf['maxspeed'] == 0, 'highway'].apply(lambda x: grouped[x])

In [9]:
# Add m/s speed and time as a function of distance and speed.

drive_edges_gdf['speed_m_s']=drive_edges_gdf['maxspeed']*1000/3600
drive_edges_gdf['drive_time_s']=drive_edges_gdf['distance']/drive_edges_gdf['speed_m_s']

## Create Shapefile for GAMA

In [10]:
# drive_edges_gdf.to_file(f"data/drive_shapefile.shp", driver='ESRI Shapefile')

### Create pandana network

In [11]:
drive_net_pdna=pandana.network.Network(
    drive_nodes['lon'], drive_nodes['lat'], 
    drive_edges_gdf['from'], drive_edges_gdf['to'], 
    drive_edges_gdf[['distance','speed_m_s','drive_time_s']], twoway=False)

In [12]:
drive_net_pdna.save_hdf5("../input_data/networks/drive_net.h5")

In [13]:
drive_nodes.to_csv("../input_data/networks/drive_nodes.csv")

In [14]:
# drive_edges_gdf.to_file('../data/drive_net.geojson', index=False)
# drive_edges_gdf.to_file('../front_end/data/drive_net.geojson', index=False)
# drive_nodes.to_csv('../data/drive_nodes.csv')

## Walk and Bike

In [16]:
walk_nodes, walk_edges = osmnet.load.network_from_bbox(
    lat_min=lat_min, lng_min=long_min, lat_max=lat_max, 
    lng_max=long_max, network_type='walk', two_way=False)

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](42.90415500,-2.62199057,43.15677470,-2.17936200);>;);out;'}"
Downloaded 44,734.0KB from www.overpass-api.de in 2.22 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](43.15445613,-2.62199057,43.40391960,-2.17600228);>;);out;'}"
Downloaded 30,778.0KB from www.overpass-api.de in 1.76 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!

In [17]:
walk_nodes=walk_nodes.rename(columns = {'x': 'lon', 'y': 'lat'})

In [18]:
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='from', right_index=True).rename(
    columns={'lon': 'from_lon', 'lat': 'from_lat'})
walk_edges=walk_edges.merge(walk_nodes, how='left', left_on='to', right_index=True).rename(
    columns={'lon': 'to_lon', 'lat': 'to_lat'})

In [19]:
walk_edges['geometry']=walk_edges.apply(
    lambda row: LineString([[row['from_lon'], row['from_lat']],
                             [row['to_lon'], row['to_lat']]]), axis=1)

In [20]:
walk_edges_gdf=gpd.GeoDataFrame(data=walk_edges, geometry='geometry', crs='epsg:4326')

In [21]:
walk_nodes.to_csv("../input_data/networks/walk_nodes.csv")

### Create walk pandana network

In [22]:
walk_net_pdna=pandana.network.Network(
    walk_nodes['lon'], walk_nodes['lat'], 
    walk_edges_gdf['from'], walk_edges_gdf['to'], 
    walk_edges_gdf[['distance']], twoway=False)

In [23]:
walk_net_pdna.save_hdf5("../input_data/networks/walk_net.h5")

In [ ]:
# walk_edges_gdf.to_file('../data/walk_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/walk_net.geojson', index=False)
# walk_nodes.to_csv('../data/walk_nodes.csv')

### Create bike pandana network

In [24]:
walk_net_pdna.save_hdf5("../input_data/networks/bike_net.h5")

In [24]:
# walk_edges_gdf.to_file('../data/bike_net.geojson', index=False)
# walk_edges_gdf.to_file('../front_end/data/bike_net.geojson', index=False)
# walk_nodes.to_csv('../data/bike_nodes.csv')